<a href="https://colab.research.google.com/github/Lokkamithran/FYP/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pwd
import os
os.chdir('/content/gdrive/My Drive/FYP_Colab/SVM')
!pwd

In [ ]:
import time

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import nltk
import pickle

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold

nltk.download('punkt')

In [ ]:
def mapping(prediction):
    # if prediction == 0:
    #     output = "not offensive."
    if prediction == 0:
        output = "offensive to other castes."
    elif prediction == 1:
        output = "offensive to handicapped people."
    elif prediction == 2:
        output = "offensive to others."
    elif prediction == 3:
        output = "racially offensive."
    elif prediction == 4:
        output = "offensive to other religions."
    elif prediction == 5:
        output = "offensive to people of the LGBTQIA+ community."
    else:
        output = "offensive to women."
    return output


In [ ]:
def calMetric(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    f1 = f1_score(y_test, pred, average='weighted')
    recall = recall_score(y_test, pred, average = 'weighted')
    precision = precision_score(y_test, pred, average = 'weighted')
    return [round(accuracy, 3), round(f1, 3), round(recall, 3), round(precision, 3)]

In [ ]:
def fit_rbf(x_train, x_test, y_train, y_test):

    start = time.process_time()
    rbf = svm.SVC(kernel = 'rbf', gamma = 1, C = 1).fit(x_train, y_train)
    end = time.process_time()
    pred = rbf.predict(x_test)

    return [calMetric(y_test, pred)]
#1, 1 - 91, 90.8

In [ ]:
def fit_poly(x_train, x_test, y_train, y_test):
    poly = svm.SVC(kernel = 'poly', degree = 4, C = 10).fit(x_train, y_train)
    pred = poly.predict(x_test)

    return calMetric(y_test, pred)

In [ ]:
# Load train and val datasets
df = pd.read_csv("../Final.csv", sep = "`")
# print(df.head())
df['Comment'].dropna(inplace=True)

df = df[df['Fine Tag'] != 'Not_offensive']

x = df["Comment"].values
y = df["Fine Tag"].values
# x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.20, random_state = 42)
# print(df['Fine Tag'].value_counts())

In [ ]:
rbf_df = pd.DataFrame(columns=['k','Test Partition No.','Accuracy','Precision','Recall','F1 Score', 'Training Time'])
poly_df = pd.DataFrame(columns=['k','Test Partition No.','Accuracy','Precision','Recall','F1 Score'])

In [ ]:
k = 5
k_fold = KFold(n_splits = k)
rbf_metrics = []
poly_metrics = []
i = 1

for training_index, testing_index in k_fold.split(x):
    x_train, x_test = x[training_index], x[testing_index]
    y_train, y_test = y[training_index], y[testing_index]

    vectorizer = TfidfVectorizer(max_features = int(1e5))
    fitted_vec = vectorizer.fit(x_train)
    x_train = fitted_vec.transform(x_train)
    x_test = fitted_vec.transform(x_test)

    encoder = LabelEncoder()
    fitted_encoder = encoder.fit(y_train)
    y_train = fitted_encoder.transform(y_train)
    y_test = fitted_encoder.transform(y_test)

    oversample = SMOTE(sampling_strategy = 'not majority')
    x_train, y_train = oversample.fit_resample(x_train, y_train)

    # rbf_results = fit_rbf(x_train, x_test, y_train, y_test)
    # rbf_acc, rbf_f1, rbf_recall, rbf_precision = rbf_results[0]
    # rbf_ttime = rbf_results[1]
    # rbf_metrics.append([rbf_acc, rbf_f1, rbf_recall, rbf_precision, rbf_ttime])
    # rbf_df.loc[len(rbf_df.index)] = [k, i, rbf_acc, rbf_precision, rbf_recall, rbf_f1, rbf_ttime]
    # print(rbf_metrics)

    poly_acc, poly_f1, poly_recall, poly_precision = fit_poly(x_train, x_test, y_train, y_test)
    poly_metrics.append([poly_acc, poly_f1, poly_recall, poly_precision])
    poly_df.loc[len(poly_df.index)] = [k, i, poly_acc, poly_precision, poly_recall, poly_f1]
    print(poly_metrics)

    print("\n\n")
    i += 1

# rbf_df.to_excel(excel_writer = r'RBF Metrics k=5.xlsx', index=False)
poly_df.to_excel(excel_writer = r'Poly Metrics k=5.xlsx', index=False)

In [ ]:
print(rbf_results[0])

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=100000)

fit = Tfidf_vect.fit(x)
# np.save('svm_x_classes.npy', fit.get_feature_names_out())

x = fit.transform(x)

encoder = LabelEncoder()
y_fit = encoder.fit(y)
y = y_fit.transform(y)

# print(encoder.classes_)
# print(encoder.transform(encoder.classes_))

oversample = SMOTE(sampling_strategy = 'not majority')
x, y = oversample.fit_resample(x, y)

# unique, frequency = np.unique(y, return_counts = True)
# print(unique)
# print(frequency)

x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.20, random_state = 42)

In [ ]:
import matplotlib.pyplot as plt

X = [20, 40, 60, 80]

plt.plot(X, acc_list, color='r', label='test-accuracy')
# plt.plot(X, f1_list, color='g', label='f1-score')

plt.xlabel("% Training Data")
plt.ylabel("F1 Score")
# plt.title("F1 Score of SVM polynomial")

plt.legend()
plt.show()

In [ ]:
print(sum(acc_list)/len(acc_list))

In [ ]:
pickle.dump(rbf, open('rbf_only_offensive.pkl', 'wb'))
pickle.dump(fit, open("tfidf.pickle", "wb"))

In [ ]:
#rbf accuracy
# rbf_pred = new_pred

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
rbf_recall = recall_score(y_test, rbf_pred, average = 'weighted')
rbf_precision = precision_score(y_test, rbf_pred, average = 'weighted')

print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))
print('Recall (RBF Kernel): ', '%.2f' % (rbf_recall*100))
print('Precision (RBF Kernel): ', '%.2f' % (rbf_precision*100))

In [ ]:
poly = svm.SVC(kernel = 'poly', degree = 4, C = 10).fit(x_train, y_train)
poly_pred = poly.predict(x_test)

#4, 10 - 77, 73

In [ ]:
pickle.dump(poly, open('poly_only_offensive.pkl', 'wb'))

In [ ]:
#Poly accuracy
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
poly_recall = recall_score(y_test, poly_pred, average = 'weighted')
poly_precision = precision_score(y_test, poly_pred, average = 'weighted')

print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1))
print('Recall (Poly Kernel): ', '%.2f' % (poly_recall))
print('Precision (Poly Kernel): ', '%.2f' % (poly_precision))

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# import pickle

# from google.colab import drive
# drive.mount('/content/gdrive')

pickled_model = pickle.load(open('poly_only_offensive.pkl', 'rb'))

Tfidf_vect_new = TfidfVectorizer(max_features=100000)
Tfidf_vect_new = pickle.load(open('tfidf.pickle', 'rb'))
# new_pred = pickled_model.predict(x_test)

# print(new_pred)

In [ ]:
poly_pred = pickled_model.predict(x_test)

In [ ]:
test_sentence = "nee laam uyir vaala ve kudathu"
test_sentence = Tfidf_vect_new.transform([test_sentence])

prediction = pickled_model.predict(test_sentence)[0]
print("This statement is", mapping(prediction))